In [ ]:
# Vitaly 
import requests
from bs4 import BeautifulSoup
import re
import json

immo_data = {
    "10663053": {},
    "10657599": {},
    "10655844": {},
    "10656608": {},
    "10654023": {},
    "10655537": {},
    "10647497": {},
    "10642679": {},
    "10132944": {},
    "10648643": {},
    "10657189": {},
    "10649460": {},
    "10643043": {},
    "10649446": {},
    "10456986": {},
    "10663298": {}
}

required_info = [
    "type", "subtype", "country", "region", "province", "district", "locality", "postalCode",
    "street", "number", "box", "constructionYear", "condition", "facadeCount", 
    "price", "netHabitableSurface", "isFurnished",
    "floor", "floorCount", "hasLift", 
    "bedroomCount", "bathroomCount", "showerRoomCount", "toiletCount",
    "hasAirConditioning", "fireplaceExists", "fireplaceCount",
    "hasGarden", "gardenSurface", "hasTerrace", "terraceSurface", "hasSwimmingPool",
    "primaryEnergyConsumptionPerSqm", "epcScore",
    "parkingCountIndoor", "parkingCountOutdoor", "parkingCountClosedBox"
]

def extractJSdata(str):
    data = dict()
    for s in required_info:
        match = re.search(r"\"" + re.escape(s) + "\":\"?([a-zA-Z _0-9]*)\"?", str)
        if match != None:
            value = match.group(1)
        else:
            value = None
        data[s] = value
    return data

data_categories = []

def scrape_property_info():
    session = requests.Session()

    for key, value in immo_data.items():
        url = "https://www.immoweb.be/en/classified/" + key
        immo_data[key]["URL"] = url

        req = session.get(url)
        status = req.status_code

        if status != 200:
            print(status)
            print(url)
            immo_data[key]["Status"] = status
        else:
            immo_data[key]["Status"] = status
            content = req.content
            s = BeautifulSoup(content, "html.parser")

            js_data = dict()

            ### to scrape from JS
            script_tags = s.find_all('script', {'type': 'text/javascript'})
            for st in script_tags:
                if st.text.find("window.classified") != -1:
                    js_var = re.search(r"\"property\":{.*\};", st.text)
                    js_text = js_var.group(0)
                    js_data = extractJSdata(js_text)
                    break
                else:
                    continue

            html_data = dict()
            for tag in s.find_all("th"):
                dataType = tag.contents[0].strip()
                dataValue = tag.findNext("td").contents[0].strip()
                if dataValue == "":
                    ### Scrape from children
                    dataValue = "???"
                html_data[dataType] = dataValue
                if dataType not in data_categories:
                    data_categories.append(dataType)

            immo_data[key]["Data"] = js_data
            immo_data[key]["Extras"] = html_data

scrape_property_info()

with open("test.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(immo_data, indent=4))

data_categories.sort()
print(len(data_categories))
for cat in data_categories:
    print(cat)

In [1]:
# Vitaly 
import requests
from bs4 import BeautifulSoup
import re
import json


def get_property_data_from_js(id, data):
    """ Add """
    # get price
    immo_data[id]["price"] = data["transaction"]["sale"]["price"]
    # get property data
    property = ["type", "subtype", "location",
                "bedroomCount", "netHabitableSurface", "building", "hasLift", "kitchen",
                "hasGarden", "gardenSurface", "hasTerrace", "terraceSurface", "land",
                "fireplaceExists", "hasSwimmingPool", "hasAirConditioning",
                "bathroomCount", "showerRoomCount", "toiletCount",
                "parkingCountIndoor", "parkingCountOutdoor", "parkingCountClosedBox"]
    for prop in property:
        if prop == "location":
            loc = ["country", "region", "province", "district", "locality", 
                   "postalCode", "street", "number", "box", "floor"]
            for l in loc:
                immo_data[id][l] = data["property"][prop][l]
        elif prop == "building":
            sub = ["constructionYear", "facadeCount", "floorCount"]
            for s in sub:
                if data["property"][prop] != None:
                    immo_data[id][s] = data["property"][prop][s]
                else:
                    immo_data[id][s] = None
        elif prop == "kitchen":
            if data["property"][prop] != None:
                immo_data[id][prop] = data["property"][prop]["type"]
            else:
                immo_data[id][prop] = None
        elif prop == "land":
            if data["property"][prop] != None:
                immo_data[id][prop] = data["property"][prop]["surface"]
            else:
                immo_data[id][prop] = None
        else:
            immo_data[id][prop] = data["property"][prop]
    # get energy consumption data
    if data["transaction"]["certificates"] != None:
        immo_data[id]["primaryEnergyConsumptionPerSqm"] = data["transaction"]["certificates"]["primaryEnergyConsumptionPerSqm"]
        immo_data[id]["epcScore"] = data["transaction"]["certificates"]["epcScore"]
    else:
        immo_data[id]["primaryEnergyConsumptionPerSqm"] = None
        immo_data[id]["epcScore"] = None
    if data["property"]["energy"] != None:
        immo_data[id]["hasDoubleGlazing"] = data["property"]["energy"]["hasDoubleGlazing"]
    else:
        immo_data[id]["hasDoubleGlazing"] = None
    sale_type = None
    if data["flags"]["isPublicSale"]:
        sale_type = "PublicSale"
    elif data["flags"]["isNotarySale"]:
        sale_type = "NotarySale"
    elif data["flags"]["isLifeAnnuitySale"]:
        sale_type = "LifeAnnuitySale"
    elif data["flags"]["isAnInteractiveSale"]:
        sale_type = "AnInteractiveSale"
    immo_data[id]["saleType"] = sale_type
    


def scrape_property_data(id, session):
    """ Add """
    url = "https://www.immoweb.be/en/classified/" + id
    immo_data[id] = dict()
    immo_data[id]["URL"] = url
    
    req = session.get(url)
    status = req.status_code

    if status != 200:
        immo_data[id]["Status"] = status
    else:
        immo_data[id]["Status"] = status
        content = req.content
        s = BeautifulSoup(content, "html.parser")
            
        script_tags = s.find_all('script', {'type': 'text/javascript'})
        for st in script_tags:
            if st.text.find("window.classified") != -1:
                js_var = re.search(r"window\.classified = (\{.*\});", st.text)
                js_var_value = js_var.group(1)
                js_data = json.loads(js_var_value)
                #immo_data[id]["Data"] = js_data
                get_property_data_from_js(id, js_data)
                break
            else:
                continue


def scrape_properties(file):
    """ Add """
    file = open(file + ".txt", "r")
    count = 0
    session = requests.Session()
    for line in file.readlines():
        id = line.strip()
        # Add concurrency
        # Send scrape_property_data to multiple workers
        if id not in immo_data.keys():
            scrape_property_data(id, session)
        count += 1
        if count % 10 == 0:
            print(count)


def save_to_json(file, data):
    """ Add """
    output = file + ".json"
    with open(output, "w", encoding="utf-8") as json_file:
        json_file.write(json.dumps(data, indent=4))
    return output


file_name = "..\data\properties_ids"
immo_data = dict()

scrape_properties(file_name)
output_file = save_to_json(file_name, immo_data)

print(f"Check {output_file} for scraped data...")


10
20
30
40
50
60
70
80
90
100
110
120
130
Check ..\data\properties_ids.json for scraped data...
